In [1]:
from PIL import Image, ImageDraw
import matplotlib.patches as patches
import math
import os
import random
from pathlib import Path
import numpy as np
import yaml
import glob
import cv2
import torch
from pathlib import Path
import pandas as pd
import os
import shutil

def myFig(img, bbox_list, drawB = True) :
    image = Image.fromarray(img,"RGB")
    draw = ImageDraw.Draw(image)

    if drawB :
        xmin = bbox[0] 
        ymin = bbox[1]
        xmax = bbox[2]
        ymax = bbox[3]

        draw.rectangle((xmin, ymin, xmax, ymax), outline=(255,0,0), width = 1) # bounding box
#     display(image)
    return image

def make_json():
    coco_json_train = {}
    coco_json_train['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
    coco_json_train['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
    coco_json_train['annotations'] = []
    coco_json_train['categories'] = [{'supercategory': 'syringe', 'id': 0, 'name': 'syringe'}]
    coco_json_train['images'] = []


    coco_json_val = {}
    coco_json_val['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
    coco_json_val['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
    coco_json_val['annotations'] = []
    coco_json_val['categories'] = [{'supercategory': 'syringe', 'id': 0, 'name': 'syringe'}]
    coco_json_val['images'] = []
    
    return coco_json_train, coco_json_val


def make_dir(path):
    if not os.path.exists(path):
        print(path)
        make_dir(os.path.dirname(path))
        os.makedirs(path, exist_ok=True)

        
def myLoadImage(im_path,img_size, resize=True, augment=True) :
    im = cv2.imread(im_path)
    print(im.shape)
    h0, w0 = im.shape[:2]
    if resize : 
        r = img_size / max(h0, w0)
        if r != 1:  # if sizes are not equal
            interp = cv2.INTER_LINEAR if augment else cv2.INTER_AREA  # random.choice(self.rand_interp_methods)
            im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
    return im, (h0, w0), im.shape[:2] 

def xywhn2xyxy(x, w=640, h=640, padw=0, padh=0):
    # Convert nx4 boxes from [x, y, w, h] normalized to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = w * (x[:, 0] - x[:, 2] / 2) + padw  # top left x
    y[:, 1] = h * (x[:, 1] - x[:, 3] / 2) + padh  # top left y
    y[:, 2] = w * (x[:, 0] + x[:, 2] / 2) + padw  # bottom right x
    y[:, 3] = h * (x[:, 1] + x[:, 3] / 2) + padh  # bottom right y
    return y

def convert_coordinates(x1, y1, x2, y2, img_width, img_height, cate):
    # 바운딩 박스의 중심 좌표와 너비, 높이 계산
    box_width = x2 - x1
    box_height = y2 - y1
    x_center = (x1 + x2) / 2.0
    y_center = (y1 + y2) / 2.0
    
    # 이미지의 너비와 높이로 나누어서 정규화된 값 계산
    x_center /= img_width
    y_center /= img_height
    box_width /= img_width
    box_height /= img_height
    
    # 결과를 문자열로 반환
    return f"{cate} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

def out_mask(mask_path):
    mask_img = cv2.imread(mask_path)
    gray = cv2.cvtColor(mask_img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)
    mask[mask==0] = 100
    mask[mask==255] = 0
    mask[mask==100] = 255
    return mask



def is_bbox_overlap(bbox1, bbox2):
    """Check if two bounding boxes overlap."""
    
    # Extract x, y, width, and height of each bbox
    x1, y1, x1_right, y1_bottom = bbox1
    x2, y2, x2_right, y2_bottom = bbox2

    # Check for overlap
    overlap = (x1 < x2_right) & (x1_right > x2) & (y1 < y2_bottom) & (y1_bottom > y2)
    
    return overlap

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from imantics import Polygons, Mask
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
import pickle
import os
import random
from rembg import remove
from rembg import new_session
import copy
mask_dict = {}
def test(back_img_list, 
         save_img_path, 
         save_label_path, 
         save_annotations_path,
         coco_json,
         target_im_files,
         id_cnt,
         t):
    category_id = 0
    for idx, back_img_file in enumerate(back_img_list) : 
        last_bbox = [] 
        back_img = cv2.imread(back_img_file)
        print(f'back_img : {back_img.shape}')
        turn = random.randrange(1,4)
        for i in range(turn) :

            s_lm = int(min(back_img.shape[0],back_img.shape[1])-100)

            target_file = target_im_files[random.randrange(0,len(target_im_files)-1)]
            target_img = cv2.imread(target_file)

            
            if target_file not in mask_dict : 
                print(f'New !!!!! target_file : {target_file}')
                out = remove(target_img, only_mask=True,post_process_mask=True)
                mask = out.copy()
                x, y, w, h = cv2.boundingRect(mask)
                mask[mask==0] = 100
                mask[mask==255] = 0
                mask[mask==100] = 255
                mask_dict[target_file] = {'mask':mask,'bbox':[x, y, w, h]}
            else :
                mask = mask_dict[target_file]['mask']
                x, y, w, h = mask_dict[target_file]['bbox']
            

            print(f'target_file : {target_file}')

            mask = mask[y:y+h,x:x+w]
            target_img = target_img[y:y+h,x:x+w]
            print('mask',mask.shape)
            
            resize_size = random.randrange(1,3)
    
            if back_img.shape[1]-50 <= mask.shape[1] or back_img.shape[0]-50 <= mask.shape[0] :
                resize_size = 2

            resize_size_w = int(mask.shape[1]/resize_size)
            resize_size_h = int(mask.shape[0]/resize_size)
            print(f'resize_size_w : {resize_size_w}')
            print(f'resize_size_h : {resize_size_h}')
            mask = cv2.resize(mask, (resize_size_w, resize_size_h), interpolation=cv2.INTER_LANCZOS4)


            target_img = cv2.resize(target_img, (resize_size_w, resize_size_h), interpolation=cv2.INTER_LANCZOS4)

            t_s = target_img.shape[0]

#             r_degree = random.randrange(0,360)

#             b = cv2.getRotationMatrix2D((t_s/2,t_s/2), r_degree,1)
#             mask = cv2.warpAffine(mask, b,(t_s,t_s),borderValue = 255)
#             target_img = cv2.warpAffine(target_img, b,(t_s,t_s),borderValue = 255)


            mask_inv = cv2.bitwise_not(mask)

            height1, width1 = target_img.shape[:2]
            height2, width2 = back_img.shape[:2]

            x_limit = int(back_img.shape[1])-resize_size_w
            y_limit = int(back_img.shape[0])-resize_size_h
            x1 = random.randrange(0,x_limit)
            y1 = random.randrange(0,y_limit)
            x2 = x1 + width1
            y2 = y1 + height1
            bbox = [x1 ,y1, x2, y2]

            flag = True
            for b_bbox in last_bbox : 
                if is_bbox_overlap(b_bbox,bbox) == True :
                    flag = False
                    break
            if flag == False :
                continue

            roi = back_img[y1:y2, x1:x2]
            fg = cv2.bitwise_and(target_img, target_img, mask=mask_inv)
            bg = cv2.bitwise_and(roi, roi, mask=mask)

            back_img[y1:y2, x1:x2] = cv2.add(fg, bg)
            last_bbox.append(bbox)
#             back_img = cv2.rectangle(back_img,(x1 ,y1),(x2, y2),(0,255,0),3) # not copying here will throw an error


#             f = plt.figure()
#             f.set_figwidth(5)
#             f.set_figheight(3)
#             plt.imshow(back_img)
#             plt.show()
    
            print(f'img : {idx+t}.jpg')
            

            height, width, _ = back_img.shape
            idx_name = idx + t
            img_res = {'license': 1, 'file_name': f'{idx_name}.jpg','coco_url': '','height': height,'width': width,'date_captured': '','flickr_url': '','id': idx_name}

            coco_json['images'].append(img_res)

            with open(f'{save_label_path}/{idx_name}.txt', "a") as f:

                x1, y1, x2, y2 = bbox[0],bbox[1],bbox[2],bbox[3]

                yolo_bbox = convert_coordinates(x1, y1, x2, y2, width, height, category_id)
                f.write(f"{yolo_bbox}\n")

                coco_bbox = [x1, y1, x2 - x1, y2 - y1]  # COCO bbox 형식으로 변환

                area = coco_bbox[2] * coco_bbox[3]
                res = {'segmentation':[] ,'area': area ,'iscrowd': 0,'image_id': idx_name,'bbox': coco_bbox ,'category_id': category_id,'id': id_cnt}
                coco_json['annotations'].append(res)
                id_cnt += 1
        cv2.imwrite(f'{save_img_path}/{idx+t}.jpg',back_img)
#         f = plt.figure()
#         f.set_figwidth(20)
#         f.set_figheight(15)
#         plt.imshow(back_img)
#         plt.show()
        
    import json
    with open(save_annotations_path, "w") as f:
        json.dump(coco_json, f)
    return id_cnt



In [3]:
root_path = '/data/ij/background_remove/coco_dataset'
if os.path.exists(root_path) : 
    shutil.rmtree(root_path)
else :
    os.mkdir(root_path)

save_img_path = f'{root_path}/images/train2017'
save_img_path_val = f'{root_path}/images/val2017'

save_label_path = f'{root_path}/labels/train2017'
save_label_path_val = f'{root_path}/labels/val2017'
save_annotations_path = f'{root_path}/annotations/'
save_annotations_train = f'{root_path}/annotations/instances_train2017.json'
save_annotations_val = f'{root_path}/annotations/instances_val2017.json'

save_path_list = [save_img_path,save_img_path_val,save_label_path,save_label_path_val,save_annotations_path]

for save_path in save_path_list : 
    make_dir(save_path)
    



/data/ij/background_remove/coco_dataset/images/train2017
/data/ij/background_remove/coco_dataset/images
/data/ij/background_remove/coco_dataset
/data/ij/background_remove/coco_dataset/images/val2017
/data/ij/background_remove/coco_dataset/labels/train2017
/data/ij/background_remove/coco_dataset/labels
/data/ij/background_remove/coco_dataset/labels/val2017
/data/ij/background_remove/coco_dataset/annotations/
/data/ij/background_remove/coco_dataset/annotations


In [4]:

target_img_root = '/data/ij/background_remove/syringe'
    
file_ex = r'.jpg'
target_im_files = np.array([i for i in os.listdir(target_img_root) if i.endswith(file_ex)], dtype = "object")
target_im_files = list(target_img_root + "/" + target_im_files)

In [5]:
file_ex = r'.jpg'
img_root_path = '/data/ij/coco_img/val2017'

im_files = np.array([i for i in os.listdir(img_root_path) if i.endswith(file_ex)], dtype = "object")
im_files = list(img_root_path + "/" + im_files)
split = int(len(im_files) * 0.8)
train_img_list = im_files[:split]
val_img_list = im_files[split:]

In [6]:
coco_json_train, coco_json_val = make_json()

In [7]:


id_cnt = 1
t = 0
id_cnt= test(train_img_list, save_img_path, save_label_path, save_annotations_train,
             coco_json_train,target_im_files,id_cnt,t)

#val
t = int(len(train_img_list))
id_cnt= test(val_img_list, save_img_path_val, save_label_path_val, save_annotations_val,
             coco_json_val,target_im_files,id_cnt,t)


back_img : (640, 480, 3)
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0440.jpg
target_file : /data/ij/background_remove/syringe/naver_0440.jpg
mask (192, 268)
resize_size_w : 268
resize_size_h : 192
img : 0.jpg
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0087.jpg
target_file : /data/ij/background_remove/syringe/naver_0087.jpg
mask (73, 337)
resize_size_w : 168
resize_size_h : 36
img : 0.jpg
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0101.jpg
target_file : /data/ij/background_remove/syringe/naver_0101.jpg
mask (291, 42)
resize_size_w : 42
resize_size_h : 291
back_img : (400, 600, 3)
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0051.jpg
target_file : /data/ij/background_remove/syringe/naver_0051.jpg
mask (261, 280)
resize_size_w : 280
resize_size_h : 261
img : 1.jpg
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0243.jpg
target_file : /data/ij/background_remove/syringe/naver_0243.jpg
mask

target_file : /data/ij/background_remove/syringe/naver_0200.jpg
mask (213, 335)
resize_size_w : 167
resize_size_h : 106
img : 20.jpg
target_file : /data/ij/background_remove/syringe/naver_0010.jpg
mask (221, 248)
resize_size_w : 248
resize_size_h : 221
img : 20.jpg
target_file : /data/ij/background_remove/syringe/naver_0130.jpg
mask (185, 260)
resize_size_w : 130
resize_size_h : 92
img : 20.jpg
back_img : (640, 512, 3)
target_file : /data/ij/background_remove/syringe/naver_0007.jpg
mask (227, 276)
resize_size_w : 276
resize_size_h : 227
img : 21.jpg
back_img : (640, 558, 3)
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0266.jpg
target_file : /data/ij/background_remove/syringe/naver_0266.jpg
mask (245, 340)
resize_size_w : 340
resize_size_h : 245
img : 22.jpg
target_file : /data/ij/background_remove/syringe/2.jpg
mask (307, 513)
resize_size_w : 256
resize_size_h : 153
img : 22.jpg
target_file : /data/ij/background_remove/syringe/naver_0440.jpg
mask (192, 268)
resize_s

target_file : /data/ij/background_remove/syringe/naver_0063.jpg
mask (243, 292)
resize_size_w : 292
resize_size_h : 243
back_img : (500, 333, 3)
target_file : /data/ij/background_remove/syringe/naver_0043.jpg
mask (97, 232)
resize_size_w : 116
resize_size_h : 48
img : 48.jpg
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0426.jpg
target_file : /data/ij/background_remove/syringe/naver_0426.jpg
mask (251, 276)
resize_size_w : 276
resize_size_h : 251
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0006.jpg
target_file : /data/ij/background_remove/syringe/naver_0006.jpg
mask (51, 273)
resize_size_w : 273
resize_size_h : 51
img : 48.jpg
back_img : (612, 612, 3)
target_file : /data/ij/background_remove/syringe/naver_0118.jpg
mask (264, 272)
resize_size_w : 272
resize_size_h : 264
img : 49.jpg
target_file : /data/ij/background_remove/syringe/naver_0199.jpg
mask (358, 243)
resize_size_w : 121
resize_size_h : 179
target_file : /data/ij/background_remove/syring

target_file : /data/ij/background_remove/syringe/naver_0073.jpg
mask (17, 12)
resize_size_w : 12
resize_size_h : 17
img : 74.jpg
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0018.jpg
target_file : /data/ij/background_remove/syringe/naver_0018.jpg
mask (105, 338)
resize_size_w : 338
resize_size_h : 105
back_img : (480, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0426.jpg
mask (251, 276)
resize_size_w : 138
resize_size_h : 125
img : 75.jpg
target_file : /data/ij/background_remove/syringe/naver_0094.jpg
mask (310, 76)
resize_size_w : 76
resize_size_h : 310
img : 75.jpg
back_img : (428, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0043.jpg
mask (97, 232)
resize_size_w : 232
resize_size_h : 97
img : 76.jpg
target_file : /data/ij/background_remove/syringe/naver_0101.jpg
mask (291, 42)
resize_size_w : 42
resize_size_h : 291
img : 76.jpg
back_img : (437, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0076.jpg
mask (300, 8

target_file : /data/ij/background_remove/syringe/naver_0095.jpg
mask (243, 71)
resize_size_w : 35
resize_size_h : 121
img : 102.jpg
New !!!!! target_file : /data/ij/background_remove/syringe/naver_0045.jpg
target_file : /data/ij/background_remove/syringe/naver_0045.jpg
mask (117, 119)
resize_size_w : 119
resize_size_h : 117
back_img : (427, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0076.jpg
mask (300, 89)
resize_size_w : 89
resize_size_h : 300
img : 103.jpg
back_img : (426, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0490.jpg
mask (245, 204)
resize_size_w : 204
resize_size_h : 245
img : 104.jpg
target_file : /data/ij/background_remove/syringe/naver_0232.jpg
mask (25, 32)
resize_size_w : 16
resize_size_h : 12
target_file : /data/ij/background_remove/syringe/naver_0117.jpg
mask (174, 316)
resize_size_w : 316
resize_size_h : 174
back_img : (480, 640, 3)
target_file : /data/ij/background_remove/syringe/2.jpg
mask (307, 513)
resize_size_w : 256
resize

...

back_img : (467, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0119.jpg
mask (213, 272)
resize_size_w : 136
resize_size_h : 106
img : 4831.jpg
target_file : /data/ij/background_remove/syringe/naver_0076.jpg
mask (300, 89)
resize_size_w : 89
resize_size_h : 300
img : 4831.jpg
back_img : (480, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0387.jpg
mask (273, 284)
resize_size_w : 284
resize_size_h : 273
img : 4832.jpg
back_img : (337, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0255.jpg
mask (280, 76)
resize_size_w : 76
resize_size_h : 280
img : 4833.jpg
target_file : /data/ij/background_remove/syringe/naver_0118.jpg
mask (264, 272)
resize_size_w : 136
resize_size_h : 132
target_file : /data/ij/background_remove/syringe/naver_0463.jpg
mask (324, 321)
resize_size_w : 160
resize_size_h : 162
img : 4833.jpg
back_img : (453, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0101.jpg
mask (291, 42)
resize_size_w : 42
resize_size

back_img : (512, 640, 3)
target_file : /data/ij/background_remove/syringe/2.jpg
mask (307, 513)
resize_size_w : 513
resize_size_h : 307
img : 4865.jpg
target_file : /data/ij/background_remove/syringe/naver_0119.jpg
mask (213, 272)
resize_size_w : 136
resize_size_h : 106
target_file : /data/ij/background_remove/syringe/naver_0118.jpg
mask (264, 272)
resize_size_w : 136
resize_size_h : 132
back_img : (480, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0073.jpg
mask (17, 12)
resize_size_w : 12
resize_size_h : 17
img : 4866.jpg
target_file : /data/ij/background_remove/syringe/naver_0426.jpg
mask (251, 276)
resize_size_w : 276
resize_size_h : 251
img : 4866.jpg
back_img : (403, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0119.jpg
mask (213, 272)
resize_size_w : 272
resize_size_h : 213
img : 4867.jpg
target_file : /data/ij/background_remove/syringe/naver_0094.jpg
mask (310, 76)
resize_size_w : 38
resize_size_h : 155
img : 4867.jpg
target_file : /data/ij/ba

back_img : (457, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0056.jpg
mask (160, 182)
resize_size_w : 182
resize_size_h : 160
img : 4900.jpg
target_file : /data/ij/background_remove/syringe/naver_0070.jpg
mask (166, 251)
resize_size_w : 251
resize_size_h : 166
back_img : (640, 424, 3)
target_file : /data/ij/background_remove/syringe/naver_0035.jpg
mask (261, 279)
resize_size_w : 139
resize_size_h : 130
img : 4901.jpg
back_img : (640, 427, 3)
target_file : /data/ij/background_remove/syringe/naver_0152.jpg
mask (215, 281)
resize_size_w : 140
resize_size_h : 107
img : 4902.jpg
target_file : /data/ij/background_remove/syringe/1.jpg
mask (287, 393)
resize_size_w : 196
resize_size_h : 143
target_file : /data/ij/background_remove/syringe/naver_0034.jpg
mask (168, 278)
resize_size_w : 278
resize_size_h : 168
img : 4902.jpg
back_img : (480, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0266.jpg
mask (245, 340)
resize_size_w : 170
resize_size_h : 122
img : 490

back_img : (427, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0034.jpg
mask (168, 278)
resize_size_w : 139
resize_size_h : 84
img : 4938.jpg
back_img : (361, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0306.jpg
mask (291, 309)
resize_size_w : 154
resize_size_h : 145
img : 4939.jpg
back_img : (500, 375, 3)
target_file : /data/ij/background_remove/syringe/naver_0333.jpg
mask (161, 214)
resize_size_w : 107
resize_size_h : 80
img : 4940.jpg
back_img : (333, 500, 3)
target_file : /data/ij/background_remove/syringe/naver_0425.jpg
mask (183, 307)
resize_size_w : 153
resize_size_h : 91
img : 4941.jpg
target_file : /data/ij/background_remove/syringe/naver_0130.jpg
mask (185, 260)
resize_size_w : 260
resize_size_h : 185
target_file : /data/ij/background_remove/syringe/naver_0268.jpg
mask (177, 274)
resize_size_w : 137
resize_size_h : 88
img : 4941.jpg
back_img : (480, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0480.jpg
mask (135, 260)
resi

back_img : (359, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0051.jpg
mask (261, 280)
resize_size_w : 280
resize_size_h : 261
img : 4968.jpg
target_file : /data/ij/background_remove/syringe/naver_0002.jpg
mask (196, 285)
resize_size_w : 285
resize_size_h : 196
back_img : (427, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0199.jpg
mask (358, 243)
resize_size_w : 121
resize_size_h : 179
img : 4969.jpg
back_img : (500, 333, 3)
target_file : /data/ij/background_remove/syringe/naver_0070.jpg
mask (166, 251)
resize_size_w : 251
resize_size_h : 166
img : 4970.jpg
target_file : /data/ij/background_remove/syringe/naver_0076.jpg
mask (300, 89)
resize_size_w : 89
resize_size_h : 300
back_img : (640, 640, 3)
target_file : /data/ij/background_remove/syringe/naver_0056.jpg
mask (160, 182)
resize_size_w : 182
resize_size_h : 160
img : 4971.jpg
target_file : /data/ij/background_remove/syringe/naver_0065.jpg
mask (246, 76)
resize_size_w : 76
resize_size_h : 246
targ

In [8]:
print('done')

done
